# Limpieza del Data

In [160]:
import pandas as pd

In [161]:
#Importamos los datas

data1= pd.read_csv('phishing_site_urls.csv') # kaggle
data2= pd.read_csv('URL dataset.csv') #data phishing

### Obtenemos informacion rapida de los datas

In [162]:
data1.info()
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 549346 entries, 0 to 549345
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   URL     549346 non-null  object
 1   Label   549346 non-null  object
dtypes: object(2)
memory usage: 8.4+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450176 entries, 0 to 450175
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   url     450176 non-null  object
 1   type    450176 non-null  object
dtypes: object(2)
memory usage: 6.9+ MB


In [163]:
# Verificar valores nulos en cada columna 
valores_nulos_data1 = data1.isnull().sum() 
valores_nulos_data2 = data2.isnull().sum() 
print("Valores nulos en data1:\n", valores_nulos_data1) 
print("\nValores nulos en data2:\n", valores_nulos_data2) 
# Verificar espacios vacíos en cada columna 
espacios_vacios_data1 = data1.applymap(lambda x: isinstance(x, str) and x.strip() == '').sum() 
espacios_vacios_data2 = data2.applymap(lambda x: isinstance(x, str) and x.strip() == '').sum()
print("\nEspacios vacíos en data1:\n", espacios_vacios_data1) 
print("\nEspacios vacíos en data2:\n", espacios_vacios_data2)

Valores nulos en data1:
 URL      0
Label    0
dtype: int64

Valores nulos en data2:
 url     0
type    0
dtype: int64


C:\Users\Marisol\AppData\Local\Temp\ipykernel_15652\3897543293.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  espacios_vacios_data1 = data1.applymap(lambda x: isinstance(x, str) and x.strip() == '').sum()
C:\Users\Marisol\AppData\Local\Temp\ipykernel_15652\3897543293.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  espacios_vacios_data2 = data2.applymap(lambda x: isinstance(x, str) and x.strip() == '').sum()



Espacios vacíos en data1:
 URL      0
Label    0
dtype: int64

Espacios vacíos en data2:
 url     0
type    0
dtype: int64


In [164]:
# Exploramos los valores unicos de las columnas 'Label' y 'type'
valores = data1['Label'].unique()
print(valores)

valores = data2['type'].unique()
print(valores)

['bad' 'good']
['legitimate' 'phishing']


In [165]:
# Eliminamos filas duplicadas 
for df in [data1, data2]:
    duplicados= df.duplicated().sum()
    print(f"Hay {duplicados} filas duplicadas")
    df.drop_duplicates(inplace=True)
    eliminado=duplicados
    print(f"Se ha eliminado {eliminado} filas")
    print(f"Datas sin filas duplicadas")


Hay 42150 filas duplicadas
Se ha eliminado 42150 filas
Datas sin filas duplicadas
Hay 0 filas duplicadas
Se ha eliminado 0 filas
Datas sin filas duplicadas


In [166]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 507196 entries, 0 to 516470
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   URL     507196 non-null  object
 1   Label   507196 non-null  object
dtypes: object(2)
memory usage: 11.6+ MB


### Aceptamos o rechazamos los conjuntos de datos?

#### Aplicamos Muestreo Aleatorio Probabilistico

n es el tamaño de la muestra.

N es el tamaño de la población.

Z es el valor Z correspondiente al nivel de confianza deseado (por ejemplo, 1.96 para un nivel de confianza del 95%).

p es la proporción estimada de la población que tiene el atributo que deseas medir (si no se conoce, comúnmente se usa 0.5).

e es el margen de error deseado (por ejemplo, 0.05 para un margen de error del 5%).

In [167]:
conteo_data1= data1['Label'].value_counts()
conteo_data2=data2['type'].value_counts()

print(conteo_data1)
print(conteo_data2)



Label
good    392897
bad     114299
Name: count, dtype: int64
type
legitimate    345738
phishing      104438
Name: count, dtype: int64


In [168]:
#Se calcula la proporcion de 'bad' y 'phishing' en las 2 datas
#proporcion data1
proporciones1= data1['Label'].value_counts(normalize=True)
propor1=proporciones1.get('bad', 0)
god= 1- propor1

#proporcion data2
proporciones2= data2['type'].value_counts(normalize=True)
propor2= proporciones2.get('phishing', 0)
legitimo= 1- propor2

print(f"la proporcion de phishing en data1 es: {propor1:2f}")
print(f"la proporcion de legitimo en data1 es: {god:2f}")
print(f"la proporcion de phishing en data2 es: {propor2:2f}")
print(f"la proporcion de legitimo en data2 es: {legitimo:2f}")


la proporcion de phishing en data1 es: 0.225355
la proporcion de legitimo en data1 es: 0.774645
la proporcion de phishing en data2 es: 0.231994
la proporcion de legitimo en data2 es: 0.768006


In [169]:
import math

#definimos parametros

# valor de Z al 95% de confianza = 1.96  valor q proporciona un buen equilibrio entre precision y tamaño de muestra
# valor de e =0.05 

Z = 1.96 #valor Z para un nivel de confianza del 95%
p1 = propor1 #proporcion estimada
e = 0.05 #margen de error deseado (5%)

#Calcular el tamaño de la muestra
n1= (Z**2 * p1 *(1-p1))/(e**2)
n1= math.ceil(n1) #redondear a numero entero
print(f"El tamaño de la muestra del data 1 es de:  {n1}")

p2= propor2
#Calcular el tamaño de la muestra
n2= (Z**2 * p2 *(1-p2))/(e**2)
n2= math.ceil(n2) #redondear a numero entero
print(f"El tamaño de la muestra del data 2 es de:  {n2}")

El tamaño de la muestra del data 1 es de:  269
El tamaño de la muestra del data 2 es de:  274


In [171]:
# Extraemos las muestras
muestra1= data1.sample(n=n1, random_state=19)
muestra2= data2.sample(n=n2, random_state= 19)
#guardamos las muestras en archivos csv
muestra1.to_csv('muestra1.csv', index=False)
muestra2.to_csv('muestra2.csv', index=False)
print("muestras guardadas, muestra1.csv y muestra2.csv")

muestras guardadas, muestra1.csv y muestra2.csv


In [213]:
import pydnsbl
import asyncio
import nest_asyncio
import validators

nest_asyncio.apply()# permite bucles de eventos anidados
resolver= pydnsbl.DNSBLIpChecker()

#funcion asincronica para verificar url
async def verificar_url(url):
    if not validators.url(url):
        print(f"URL invalida: {url}")
        return "invalid"
    try:
        result =await resolver.check_async(url)
        if result.blacklisted:
            return "bad"
        else:
            return "god"
    except ValueError as e:
        print(f" Error verificando URL {url}: {e}")
        return "unknowm"
# Función para comparar y contar las discrepancias 
async def comparar_resultados(muestra, URL_column, Label_column):
    discrepancias = 0 
    for index, row in muestra.iterrows(): 
        url = row[URL_column] 
        tipo = row[Label_column] 
        bad = await verificar_url(url) 
# Comprobar si la verificación coincide con el tipo 
        if (bad == "bad" and tipo == 'god') or (bad=="god" and tipo =='bad'):
            discrepancias += 1 
    return discrepancias 
# Comparar resultados para la muestra de data1 
discrepancias_data1 = asyncio.run(comparar_resultados(muestra1, 'URL', 'Label')) 
# Imprimir los resultados 
print(f"Discrepancias en data1: {discrepancias_data1}")



URL invalida: kuklaskorner.com/index.php/hockey/tags/tag/michel+therrien
URL invalida: histopolis.com/Place/US/NY/St_Lawrence_County/Massena_Center
URL invalida: profesionalisti.com/hot-video-pro/dan4o-piseca-youtube-66990.html
URL invalida: postalzipcodeof.com/CA-_Canada/QC-Quebec/Anjou_West/
URL invalida: www.mycutelittle.iamspace.com/gril/to.htm?c=www.megaupload.com/?d=7Y12ESN9
URL invalida: naia.cstv.com/sports/m-baskbl/spec-rel/032308aak.html
URL invalida: melodicsoundwaves.com/2011/11/album-review-keeping-secrets-last.html
URL invalida: www.mobileer.com/index.html
URL invalida: my-akshaykumar.blogspot.com/
URL invalida: lyngsat-logo.com/download.html?imagename=Musimax&imagepath=mm/musimax
URL invalida: kmresults.com/solutions/view?id=541
URL invalida: en.wikipedia.org/wiki/Category:Populated_coastal_places_in_Japan
URL invalida: www.corcel22.com.br//aulas/baviso.php
URL invalida: travelingluck.com/North%20America/Mexico/Chihuahua/_3983312_Saucillo.html
URL invalida: wn.com/List_o

# Concatenamos datas

In [230]:
#renombrar columnas 
data1.rename(columns={'URL': 'url', 'Label': 'status'}, inplace=True)
data2.rename(columns={'type': 'status'}, inplace=True)
#actualizamos las etiquetas del data1
data1['status']=data1['status'].replace({'bad': 'phishing', 'good': 'legitimate'})

#unir datas
datas=pd.concat([data1,data2], ignore_index=True)
print("Las datas han sido unidas")
print(datas.columns)

Las datas han sido unidas
Index(['url', 'status'], dtype='object')


In [231]:
datas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 957372 entries, 0 to 957371
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   url     957372 non-null  object
 1   status  957372 non-null  object
dtypes: object(2)
memory usage: 14.6+ MB


In [232]:
conteo_datas= datas['status'].value_counts()
print(conteo_datas)


status
legitimate    738635
phishing      218737
Name: count, dtype: int64


In [ ]:
import string

# Función para estandarizar las urls 
def remove_prefixes(url): 
    prefixes=['https://', 'http://', 'ftp://', 'www.']
    for prefix in prefixes: 
        if url.startswith(prefix):
            return url[len(prefix):] 
        return url
datas['url'] = datas['url'].apply(remove_prefixes)

#Se añade columnas
#definir el alfabeto y otros caracteres
alfabeto= string.ascii_lowercase
caracteres="!@#$%^&*()_+-=[]{}|;:'\",.<>?/"
numeros="0123456789"
#funcion para contar las ocurrecnias de cada caracter
def count_chars(url,chars):
    return{chars:url.count(char)for char in chars}
#añadir columna longitus de la url
datas['url_lenght']=datas['url'].apply(len)
#añadir numero de caracteres especiales
datas['num_special_chars']=datas['url'].apply(lambda x: sum(x.count(c) for c in caracteres))
#añadir columnas para cada letra de alfabeto.
for char in alfabeto:
    datas[char]=datas['url'].apply(lambda x: x.count(char))
#añadir columnas para cada numeor
for num in numeros:
    datas[num]=datas['url'].apply(lambda x: x.count(num))
#añadir columnas para caracteres
for char in caracteres:
    datas[char]=datas['url'].apply(lambda x: x.count(char))
# Añadir columna de número de subdominios 
datas['num_subdomains'] = datas['url'].apply(lambda x: x.count('.')) 
# Añadir columna para presencia de HTTPS 
datas['has_https'] = datas['url'].apply(lambda x: 1 if 'https' in x else 0) 
# Añadir columna para presencia de IP en el dominio 
import re 
datas['has_ip'] = datas['url'].apply(lambda x: 1 if re.match(r'http[s]?://\d+\.\d+\.\d+\.\d+', x) else 0) 

print(datas.head())    

                                                 url    status   a  b   c   d  \
0  nobell.it/70ffb52d079109dca5664cce6f317373782/...  phishing  10  4  13  10   
1  www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...  phishing   4  2   7   4   
2  serviciosbys.com/paypal.cgi.bin.get-into.herf....  phishing   7  2  11   5   
3  mail.printakid.com/www.online.americanexpress....  phishing   4  0   3   2   
4  thewhiskeydregs.com/wp-content/themes/widescre...  phishing   0  0   5   3   

    e  f  g  h  ...  1  2  3  4   5  6  7  8  9  has_ip  
0  13  9  6  2  ...  6  6  7  3   6  7  9  2  4       0  
1   3  1  4  3  ...  1  0  0  0   0  0  0  0  0       0  
2  16  8  2  5  ...  4  7  7  6  10  7  3  2  1       0  
3   5  0  0  1  ...  0  0  0  0   0  0  0  0  0       0  
4  13  0  1  3  ...  0  4  0  5   0  0  6  6  0       0  

[5 rows x 72 columns]


In [241]:
datas.to_csv('datas.csv', index=False)